## Big Data Analytics

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [ ]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab


# Part 2.1: Introduction to Pandas

[Pandas](http://pandas.pydata.org/) is an open-source library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.  It is based on the [dataframe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) concept found in the [R programming language](https://www.r-project.org/about.html).  

The data frame is a crucial component of Pandas.  We will use it to access the [auto-mpg dataset](https://archive.ics.uci.edu/ml/datasets/Auto+MPG).  You can find this dataset on the UCI machine learning repository.  

UCI took this dataset from the StatLib library, which Carnegie Mellon University maintains. The dataset was used in the 1983 American Statistical Association Exposition.  It contains data for 398 cars, including [mpg](https://en.wikipedia.org/wiki/Fuel_economy_in_automobiles), [cylinders](https://en.wikipedia.org/wiki/Cylinder_(engine)), [displacement](https://en.wikipedia.org/wiki/Engine_displacement), [horsepower](https://en.wikipedia.org/wiki/Horsepower) , weight, acceleration, model year, origin and the car's name.

The following code loads the MPG dataset into a data frame:

In [ ]:
!pip install googledrivedownloader

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

# Glass data from https://archive.ics.uci.edu/ml/machine-learning-databases/glass/
gdd.download_file_from_google_drive(file_id='1Eu7nSAOVyqKEMhRSLekCjIgxZLW-sjN2',
                                    dest_path='/content/auto-mpg.csv')

In [ ]:
# Simple dataframe
import os
import pandas as pd

df = pd.read_csv("/content/auto-mpg.csv")
print(df[0:5])

The **display** function provides a cleaner display than merely printing the data frame.  Specifying the maximum rows and columns allows you to achieve greater control over the display.

In [ ]:
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)
display(df)

It is possible to generate a second data frame to display statistical information about the first data frame.

In [ ]:
# Strip non-numerics
df = df.select_dtypes(include=['int', 'float'])

headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'name' : field,
        'mean': df[field].mean(),
        'var': df[field].var(),
        'sdev': df[field].std()
    })

for field in fields:
    print(field)

This code outputs a list of dictionaries that hold this statistical information.  This information looks similar to the JSON code.  To as proper JSON, the program should add these records to a list and call the Python JSON library's **dumps** command called.

The Python program can convert this JSON-like information to a data frame for better display.

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
df2 = pd.DataFrame(fields)
display(df2)

## Missing Values

Missing values are a reality of machine learning.  Ideally, every row of data will have values for all columns.  However, this is rarely the case.  Most of the values are present in the MPG database.  However, there are missing values in the horsepower column.  A common practice is to replace missing values with the median value for that column.  The program calculates the median as described [here](https://www.mathsisfun.com/median.html).  The following code replaces any NA values in horsepower with the median:

In [ ]:
import os
import pandas as pd

df = pd.read_csv(
    "/content/auto-mpg.csv", 
    na_values=['NA', '?'])
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")
    
print("Filling missing values...")
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)
# df = df.dropna() # you can also simply drop NA values
                 
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")

# Dealing with Outliers

Outliers are values that are unusually high or low.  Sometimes outliers are simply errors; this is a result of [observation error](https://en.wikipedia.org/wiki/Observational_error).  Outliers can also be truly large or small values that may be difficult to address. We typically consider outliers to be a value that is several standard deviations from the mean.  The following function can remove such values.    

In [ ]:
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)

The code below will drop every row from the Auto MPG dataset where the horsepower is more than two standard deviations above or below the mean.

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

df = pd.read_csv(
    "/content/auto-mpg.csv",
    na_values=['NA','?'])

# create feature vector
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)

# Drop the name column
df.drop('name',1,inplace=True)

# Drop outliers in horsepower
print("Length before MPG outliers dropped: {}".format(len(df)))
remove_outliers(df,'mpg',2)
print("Length after MPG outliers dropped: {}".format(len(df)))

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)
display(df)

## Dropping Fields

Some fields are of no value to the neural network should be dropped.  The following code removes the name column from the MPG dataset.

In [ ]:
import os
import pandas as pd

df = pd.read_csv(
    "/content/auto-mpg.csv",
    na_values=['NA','?'])

print(f"Before drop: {list(df.columns)}")
df.drop('name', 1, inplace=True)
print(f"After drop: {list(df.columns)}")

## Concatenating Rows and Columns
Python can concatenate rows and columns together to form new data frames.  The code below creates a new data frame from the **name** and **horsepower** columns from the Auto MPG dataset.  The program does this by concatenating two columns together.

In [ ]:
# Create a new dataframe from name and horsepower

import os
import pandas as pd

df = pd.read_csv(
    "/content/auto-mpg.csv",
    na_values=['NA','?'])

col_horsepower = df['horsepower']
col_name = df['name']
result = pd.concat([col_name, col_horsepower], axis=1)

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)
display(result)

The **concat** function can also concatenate two rows together.  This code concatenates the first two rows and the last two rows of the Auto MPG dataset.

In [ ]:
# Create a new dataframe from first 2 rows and last 2 rows

import os
import pandas as pd

df = pd.read_csv(
    "/content/auto-mpg.csv",
    na_values=['NA','?'])

result = pd.concat([df[0:2],df[-2:]], axis=0)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 0)
display(result)

You might wish only to convert some of the columns, to leave out the name column, use the following code.

## Saving a Dataframe to CSV

 The following code performs a shuffle and then saves a new copy.

In [ ]:
import os
import pandas as pd
import numpy as np

path = "."

df = pd.read_csv(
    "/content/auto-mpg.csv",
    na_values=['NA','?'])

filename_write = os.path.join(path, "auto-mpg-shuffle.csv")
df = df.reindex(np.random.permutation(df.index))
# Specify index = false to not write row numbers
df.to_csv(filename_write, index=False) 
print("Done")